#### In this notebook, assuming an effective size of 15 (one present always) species, we will look at CS (BPDN,ADMM) on effectively 187 points. For each species, we will only look at predictions on the points where the species was present to begin with. (this is the first set of species that maximises data points)

#### import the relevant files and modules

In [1]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import random
import time

from sporco.admm import bpdn
### using the ADMM algorithm 
### we can also use the PGM algorithm 


from scipy.linalg import hadamard


from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

import time 
from scipy import stats

iHiV = pd.read_pickle("~/bge-analysis-simv3/iHiV.pkl")
iHiVred = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/ophelli-ryan/iHiV15.pkl")

import warnings
warnings.filterwarnings("ignore")

In [2]:
X = pd.read_pickle("~/bge-analysis-simv3/X15.pkl")

In [3]:
opt = bpdn.BPDN.Options({'Verbose': False, 'MaxMainIter': 500,
                         'RelStopTol': 1e-8, 'AutoRho': {'RsdlTarget': 1.0}})

#### read the effective species presence-absence

In [4]:
pa_redv1 = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/ophelli-ryan/eff-16-list1-red-pa.pkl")

In [5]:
labdl = np.round(np.logspace(-5,0.8,5),5)
lam = list(labdl)
lam.append([0.5,0.6,1.0])
lamf = list(pd.DataFrame(lam)[0].explode().values)
lamf.sort()
lamf = lamf[:7]
lamf

[1e-05, 0.00028, 0.00794, 0.22387, 0.5, 0.6, 1.0]

In [6]:
random_state = None
laml = lamf

In [7]:
#pa_redv1

In [8]:
B = pd.read_pickle("~/bge-analysis-simv3/B.pkl")
A = pd.read_pickle("~/bge-analysis-simv3/A.pkl")

tableg=A.T
species_onesall = [] 
for species in range(16):
    species_onesall.append(tableg[tableg[species]==1.0][species].index.to_list())
species_zerosall = []
for species in range(16):
    species_zerosall.append(tableg[tableg[species]==0.0][species].index.to_list())

##### compile the well numbers for each species for its presence 

In [9]:
species_ones = []
for i in range(16):
    species_ones.append(pa_redv1[pa_redv1[i]==1.0]["well number"].values)

In [10]:
ones_len = [] 
for ii in range(len(species_ones)):
    ones_len.append(len(species_ones[ii])) 

In [11]:
steadystate = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/ophelli-ryan/eff-16-list1-red-sst-REL.pkl")
sst = steadystate.T

In [12]:
bias_var_listm = [3,3,3,3,3,3,3,3,5,3,5,3,3,7,5,7]

In [ ]:
err = []
errlin = []
samp = 100
for species in range(8,16):
    cv_splits = bias_var_listm[species]
    subsetall = A[species_onesall[species]].T
    subsetall["indexinwhole"] = subsetall.index
    subsetall.index = np.arange(0,2**15,1)
    givendata = subsetall[subsetall["indexinwhole"].isin(species_ones[species])]
    m = int(samp*(ones_len[species])/100)
    cvdata = m
    ri = random.sample(list(givendata.index),m)
    ri.sort() 
    sstgiven = steadystate.T[subsetall[subsetall["indexinwhole"].isin(species_ones[species])]["indexinwhole"].values].T
    sstgiven.index = givendata.index
    y2 = sstgiven.T[ri].T[species]
    #y2 = sst[ri].T[species]
    data_present = sstgiven.T[ri].T.index
    xs = list(data_present)
    random.shuffle(xs)
    for iteras in range(5):
        kf = KFold(n_splits=cv_splits, random_state=iteras*10, shuffle=True)
        kf.get_n_splits(data_present)
        for lmda in laml:
            for train_index, test_index in kf.split(data_present):
                startt = time.time()
                X_train, X_test = data_present[train_index], data_present[test_index]
                rinew = list(X_train)
                rileft = list(X_test)
                D = iHiVred[rinew,:]
                y3 = y2[rinew]
                s2 = np.array([y3.values])
                s3 = s2.T
                b = bpdn.BPDN(D, s3, lmda, opt)
                x = b.solve()
                yrecon = np.dot(iHiVred,x.ravel())
                #print(time.time()-startt,iteras,"cs")
                err.append([iteras,lmda,species,yrecon[rileft],rileft,rinew,cv_splits,len(rileft),len(rinew)])
                startt = time.time()
                D = X[rinew,:]
                y3 = y2[rinew]
                s2 = np.array([y3.values])
                s3 = s2.T
                b = bpdn.BPDN(D, s3, lmda, opt)
                x = b.solve()
                yrecon = np.dot(X,x.ravel())
                #print(time.time()-startt,iteras,"linreg")
                errlin.append([iteras,lmda,species,yrecon[rileft],rileft,rinew,cv_splits,len(rileft),len(rinew)])


In [ ]:
errdf  = pd.DataFrame(err)
errdflin = pd.DataFrame(errlin)

In [ ]:
pd.to_pickle(errdf,"16l1-"+str(species)+"-bge.pkl")
pd.to_pickle(errdflin,"16l1-"+str(species)+"-lin.pkl")

In [ ]:
pd.read_pickle("16l1-"+str(species)+"-bge.pkl")
pd.read_pickle("16l1-"+str(species)+"-lin.pkl")

In [ ]:
c

In [1]:
import pandas as pd
pd.read_pickle("~/compressed_sensingv1/realdatasets/OV-v2/2pow15-matrices/16l1-7-lin.pkl")

,0,1,2,3,4,5,6,7,8
0,0,0.00001,0,"[0.9142107008370788, 0.45360880709973733, 0.74...","[1, 260, 4226, 4228, 4256, 8197, 24581]","[0, 2, 4, 20, 32, 128, 132, 4102, 4230, 8199, ...",3,7,14
1,0,0.00001,0,"[0.9981190840081415, 0.6563016788958447, 0.973...","[2, 20, 32, 128, 8325, 12420, 16644]","[0, 1, 4, 132, 260, 4102, 4226, 4228, 4230, 42...",3,7,14
2,0,0.00001,0,"[0.46512738421946254, 0.1819860220116709, 0.64...","[0, 4, 132, 4102, 4230, 8199, 12294]","[1, 2, 20, 32, 128, 260, 4226, 4228, 4256, 819...",3,7,14
3,0,0.00028,0,"[0.9137556052895592, 0.4533387786154435, 0.743...","[1, 260, 4226, 4228, 4256, 8197, 24581]","[0, 2, 4, 20, 32, 128, 132, 4102, 4230, 8199, ...",3,7,14
4,0,0.00028,0,"[0.9964990840812247, 0.6568416789796441, 0.968...","[2, 20, 32, 128, 8325, 12420, 16644]","[0, 1, 4, 132, 260, 4102, 4226, 4228, 4230, 42...",3,7,14
...,...,...,...,...,...,...,...,...,...
835,4,0.60000,7,"[0.28202509888596816, 0.4215674616744828, 0.36...","[1, 4, 160, 161, 544, 16388]","[0, 2, 32, 33, 38, 112, 128, 162, 672, 1185, 1...",3,6,12
836,4,0.60000,7,"[0.27881359055489674, 0.27881359055489674, 0.2...","[2, 33, 38, 112, 128, 1185]","[0, 1, 4, 32, 160, 161, 162, 544, 672, 12545, ...",3,6,12
837,4,1.00000,7,"[0.25290598127894326, 0.25290598127894326, 0.2...","[0, 32, 162, 672, 12545, 24580]","[1, 2, 4, 33, 38, 112, 128, 160, 161, 544, 118...",3,6,12
838,4,1.00000,7,"[0.3196328856137202, 0.3196328856137202, 0.319...","[1, 4, 160, 161, 544, 16388]","[0, 2, 32, 33, 38, 112, 128, 162, 672, 1185, 1...",3,6,12
